In [ ]:
import os
import pandas as pd
import faiss
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

output_dir = 'vectordb_2/categorywise/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

model = AutoModel.from_pretrained('BM-K/KosimCSE-bert-multitask').cuda()
tokenizer = AutoTokenizer.from_pretrained('BM-K/kosimCSE-bert-multitask')

def get_embeddings_in_batches(texts, batch_size=8192):
    dataset = TextDataset(texts)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    embeddings_list = []
    model.eval()

    with torch.no_grad():
        for batch_texts in tqdm(data_loader, desc="Processing batches", leave=True):
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to('cuda')

            with torch.cuda.amp.autocast(): 
                outputs = model(**inputs).last_hidden_state[:, 0, :]  
                embeddings = outputs.cpu().numpy()

            embeddings_list.append(embeddings)

    return np.vstack(embeddings_list)

/home/jaesung/anaconda3/envs/pm1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser   #출력을 파싱하여 문자열 형태로 만드는 클래스
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  #langchain_core.prompts로부터 프롬프트를 관리하는 클래스를 가져옮
from langchain.chat_models import ChatOpenAI


# llm = ChatOllama(model="EEVE-Korean-10.8B:latest")
llm = ChatOpenAI(
        api_key="ollama",
        # model="EEVE-Korean-10.8B:latest",
        model="bnksys/eeve:10.8b-korean-instruct-q8-v1",
        # model="bnksys/eeve:10.8b-korean-instruct-f16-v1",
        base_url="http://localhost:11434/v1",
        temperature=0,
        streaming=True    
    ) 

prompt_prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

chain = prompt_prompt | llm | StrOutputParser()

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI Assistant. You must answer in Korean.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chat_chain = chat_prompt | llm | StrOutputParser()

In [ ]:
import os
import pandas as pd
import faiss

def find_similar(query):
    category_dir = 'vectordb_2/categorywise'

    first_element = query.split(',')[0].strip()  

    index_file = os.path.join(category_dir, f"{first_element}_faiss_index.bin")
    metadata_file = os.path.join(category_dir, f"{first_element}_metadata.csv")

    if os.path.exists(index_file) and os.path.exists(metadata_file):
        index = faiss.read_index(index_file)
        meta_df = pd.read_csv(metadata_file)
        metadata = meta_df.apply(lambda row: ', '.join(row.astype(str)), axis=1).tolist()

        # meta_df.columns = ['original']
        # metadata = meta_df['original'].tolist()

        query_embedding = get_embeddings_in_batches([query])

        D, I = index.search(query_embedding, k=1)

        return metadata[I[0][0]]
    else:
        all_index_files = [f for f in os.listdir(category_dir) if f.endswith('_faiss_index.bin')]
        if all_index_files:
            for idx_file in all_index_files:
                file_base = idx_file.replace('_faiss_index.bin', '')
                index_file = os.path.join(category_dir, f"{file_base}_faiss_index.bin")
                metadata_file = os.path.join(category_dir, f"{file_base}_metadata.csv")

                if os.path.exists(index_file) and os.path.exists(metadata_file):
                    index = faiss.read_index(index_file)
                    meta_df = pd.read_csv(metadata_file)
                    metadata = meta_df.apply(lambda row: ', '.join(row.astype(str)), axis=1).tolist()
                    
                    # meta_df.columns = ['original']
                    # metadata = meta_df['original'].tolist()

                    query_embedding = get_embeddings_in_batches([query])

                    D, I = index.search(query_embedding, k=1)

                    if I[0][0] != -1:  #
                        return metadata[I[0][0]]

        return None


In [11]:
def gen_response(query, most_similar, category):
    
    instruction = f"""
    너는 식품 개발자야.
    """
    # 1. 배합비를 생성해줘. 이 때, 배합비를 생성할 때 {query.split(', ')[1:]}은 조합을 헤치지 않는 이상 포함되어야 해.
    # 2. 배합비의 합이 100%가 되게 재계산 해줘. 
    # 3. 출력 형식은 example들과 반드시 같도록 수정해줘.
    # # Q와 A를 모두 제공하는 예시 (형식 강화)
    example_1 = """Q: 원본 식재료 ['토마토', '아보카도', '적양파', '바질']의 카테고리는 소스류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
    A: 레드 벨 페퍼: 40%, 아보카도: 30%, 적양파: 20%, 파슬리: 10%"""

    example_2 = """Q: 원본 식재료 ['올리브 오일', '발사믹 식초', '레몬즙']의 카테고리는 소스류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
    A: 올리브 오일: 50%, 레드 와인 식초: 30%, 시트러스 주스: 20%"""

    example_3 = """Q: 원본 식재료 ['감자', '당근', '양파', '마늘', '파슬리', '버터']의 카테고리는 스튜야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
    A: 감자: 35%, 당근: 20%, 샬롯: 15%, 생강: 10%, 고수: 10%, 올리브 오일: 10%"""

    example_4 = """Q: 원본 식재료 ['돼지고기', '간장', '설탕', '생강', '파', '마늘']의 카테고리는 조림류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
    A: 닭고기: 40%, 간장: 20%, 꿀: 15%, 생강: 10%, 샬롯: 10%, 미림: 5%"""

    example_5 = """Q: 원본 식재료 ['쌀', '김', '참기름', '계란', '간장', '소금']의 카테고리는 밥류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
    A: 현미: 30%, 김: 15%, 들기름: 20%, 메추리알: 10%, 간장: 15%, 소금: 10%"""

    # 사용자의 입력
    user_input = f"""Q: 원본 식재료 {most_similar}의 카테고리는 {category}야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100% 로 맞춰줘."""

    
    response = llm.invoke(f"{instruction}\n{example_1}\n{example_2}\n{example_3}\n{example_4}\n{example_5}\n{user_input}")

    return response

In [12]:
def gen_response(query, most_similar, category):
    instruction = """
    너는 식품 개발자야. 다음 순서대로 빠르게 배합비를 생성해줘:

    1. 카테고리에 대한 식품을 만들기 위한 메인 재료를 필수적으로 포함해.
    2. 추가 재료에서 식재료를 추가해.
    3. 배합비의 합이 100%가 되도록 배합비를 완성해
    4. 불필요한 정보는 무시하고, 배합비만 간결하게 제시해줘.

    이 지침을 따르면서 응답 시간을 최소화해줘.
    """

    example_1 = f"Q: 샐러드를 만들고 있어. 상추, 토마토, 오이, 드레싱과 케일, 방울토마토, 애호박, 발사믹 식초를 참고해서 배합비를 생성해줘. A: 상추: 30%, 방울토마토: 25%, 오이: 15%, 애호박: 10%, 발사믹 식초: 10%, 케일: 10%"
    example_2 = f"Q: 스파게티를 만들고 있어. 면, 토마토 소스, 올리브 오일, 바질과 페투치니, 마리나라 소스, 해바라기유, 파슬리를 참고해서 배합비를 생성해줘. A: 면: 40%, 마리나라 소스: 30%, 해바라기유: 15%, 파슬리: 10%, 바질: 5%"
    # example_3 = f"Q: 볶음밥을 만들고 있어. 쌀, 당근, 달걀, 간장과 현미, 브로콜리, 치킨, 된장을 참고해서 배합비를 생성해줘. A: 쌀: 35%, 현미: 30%, 치킨: 20%, 브로콜리: 10%, 된장: 5%"
    # example_4 = f"Q: 스무디를 만들고 있어. 바나나, 딸기, 요거트, 꿀과 망고, 블루베리, 두유, 아가베 시럽을 참고해서 배합비를 생성해줘. A: 바나나: 30%, 딸기: 25%, 망고: 20%, 블루베리: 15%, 두유: 5%, 아가베 시럽: 5%"
    # example_5 = f"Q: 찜닭을 만들고 있어. 닭고기, 감자, 양파, 당면과 오리고기, 고구마, 대파, 당면을 참고해서 배합비를 생성해줘. A: 닭고기: 40%, 감자: 25%, 오리고기: 15%, 당면: 10%, 대파: 5%, 고구마: 5%"

    user_input = f"""Q: {category}의 요리 만들고 있어.메인 재료 {', '.join(query.split(', ')[1:])}와 추가 재료 {most_similar}를 참고해서 배합비를 생성해줘."""
    
    response = llm.invoke(f"{instruction}\n{example_1}\n{example_2}\n{user_input}")

    return response

In [13]:
def gen_response_v2(A, missed_q):
    instruction = """
    너는 식품 개발자야. 주어진 배합비에 빠진 재료를 추가하고, 배합비의 합이 100%가 되게 조정해줘.
    
    1. 주어진 재료와 배합비를 유지하면서, 추가된 재료를 포함하여 배합비를 다시 조정해야 해.
    2. 출력 형식은 '재료명: 비율%'의 형태로, 각 재료마다 비율을 정확히 기입해야 해.
    3. 불필요한 설명 없이, 재료와 비율만 출력해. 그 외에 설명은 전혀 포함하지 말 것.
    4. 예시는 반드시 '재료명: 비율%' 형태를 따르고, 형식을 지켜서 출력해.
    """

    example = f"""Q: 참치: 30%, 밥: 45%, 다시마: 5%, 김: 10%, 정제소금: 5%, 냉동다진마늘: 2%, 풋고추풍미료p: 1%, 이온엿물엿: 1%, 표고무밑국물: 1%, 5리보뉴클레오티드이나트륨: 1%, 당근: 2%, 마늘쫑: 2%, 절임무: 2%, 조미김: 2%, 절임우엉: 2%, 지리멸치: 2%, 참기름: 2%, 대두유: 2%, 냉동청양고추: 2%, 두부곤약쌀: 2%, 양조간장: 2%의 배합비에 '마라'를 추가하여 배합비를 다시 생성해줘.
    A: 참치: 28%, 밥: 42%, 마라: 5%, 다시마: 5%, 김: 8%, 정제소금: 4%, 냉동다진마늘: 2%, 풋고추풍미료p: 1%, 이온엿물엿: 1%, 표고무밑국물: 1%, 5리보뉴클레오티드이나트륨: 1%, 당근: 1%, 마늘쫑: 1%, 절임무: 1%, 조미김: 1%, 절임우엉: 1%, 지리멸치: 1%, 참기름: 1%, 대두유: 1%, 냉동청양고추: 1%, 두부곤약쌀: 1%, 양조간장: 1%.
    """

    user_input = f"Q: {A}에 {', '.join(missed_q)}를 추가해서 배합비를 다시 생성해줘. 오직 '재료명: 비율%' 형식으로 출력해."

    response = llm.invoke(f"{instruction}\n{example}\n{user_input}")

    return response





In [18]:
import re
# query = "떡류, 맵쌀, 가당백앙금, 냉동쑥"
# query = "유음료, 원유, 정제수, 가는정백당"
# query = "빵류, 딸기, 생크림"
# query = "빵류, 전란액, 하얀설탕, 박력분"
query = "유음료, 김치, 돼지고기, 마라"
category = query.split(', ')[0].strip()


most_similar = find_similar(query)
print(most_similar)

response = gen_response(query, most_similar, category)
print(response)

# 
response = response.content.split('A:')[-1].strip()
matches = re.findall(r'[가-힣a-zA-Z0-9]+:\s*\d+%', response)
A = ', '.join(matches)

missed_query = []
for q in query.split(', ')[1:]:
    if q.strip() not in A:
        missed_query.append(q)

print(A)
print(query)
print(missed_query)

Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

정제염, 볶음쌀가루, 볶음귀리분말, 내츄락dp, 참깨페이스트, 볶음통밀분말, 가는정백당, 볶은콩가루, 생크림, 볶음현미분말, 볶은퀴노아분말, 렌틸콩볶음분말, 정제수, 원유
content='A: 김치: 30%, 돼지고기: 25%, 참깨페이스트: 10%, 내츄락스: 10%, 생크림: 10%, 정제수: 10%, 볶음쌀가루: 5%, 볶음귀리분말: 5%, 볶음통밀분말: 5%, 가는정백당: 5%, 볶은콩가루: 5%, 볶은현미분말: 5%, 볶은퀴노아분말: 5%, 렌틸콩볶음분말: 5%' additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-b8e7a176-85ea-4378-b9a2-b8734ca75d7d-0'
김치: 30%, 돼지고기: 25%, 참깨페이스트: 10%, 내츄락스: 10%, 생크림: 10%, 정제수: 10%, 볶음쌀가루: 5%, 볶음귀리분말: 5%, 볶음통밀분말: 5%, 가는정백당: 5%, 볶은콩가루: 5%, 볶은현미분말: 5%, 볶은퀴노아분말: 5%, 렌틸콩볶음분말: 5%
유음료, 김치, 돼지고기, 마라
['마라']


In [39]:
query_li = [
 '밥류, 김치, 돼지고기, 마라',
 '밥류, 간장, 달걀, 대파',
 '밥류, 된장, 시금치, 취두부',
 '밥류, 참치, 다시마, 김',
 '밥류, 고추장, 버섯, 닭고기',
 '국물류, 된장, 감자, 잔탄검',
 '국물류, 소고기, 무, 파',
 '국물류, 조개, 미역',
 '국물류, 고추장, 두부, 대파',
 '국물류, 멸치, 다시마, 무',
 '농산물, 감자, 양파, 당근',
 '농산물, 상추, 오이, 고추장',
 '농산물, 배추, 마늘, 고춧가루',
 '농산물, 시금치, 간장, 깨',
 '농산물, 무, 고추가루, 설탕',
 '두부, 두부, 간장, 파',
 '두부, 두부, 김치, 돼지고기',
 '두부, 두부, 참기름, 깨',
 '두부, 두부, 된장, 버섯',
 '두부, 두부, 달걀, 파',
 '디저트, 초콜릿, 우유, 설탕',
 '디저트, 버터, 설탕, 밀가루',
 '디저트, 계란, 우유, 딸기',
 '디저트, 밀가루, 바나나, 계란',
 '디저트, 생크림, 설탕, 과일',
 '만두, 돼지고기, 배추, 부추',
 '만두, 두부, 부추, 당면',
 '만두, 김치, 돼지고기, 양파',
 '만두, 새우, 돼지고기, 마늘',
 '만두, 소고기, 대파, 당면',
 '면, 돼지고기, 된장, 면',
 '면, 고추장, 참기름, 소면',
 '면, 해물, 파, 칼국수 면',
 '면, 간장, 마늘, 우동면',
 '면, 새우, 고추, 라면',
 '소스류, 간장, 고춧가루, 마늘',
 '소스류, 된장, 참기름, 깨',
 '소스류, 고추장, 설탕, 식초',
 '소스류, 마요네즈, 케첩, 레몬즙',
 '소스류, 간장, 설탕, 생강',
 '수산물, 새우, 마늘, 버터',
 '수산물, 고등어, 간장, 무',
 '수산물, 명태, 고춧가루, 파',
 '수산물, 조개, 된장, 다시마',
 '수산물, 오징어, 고추장, 양파',
 '음료, 커피, 우유, 설탕',
 '음료, 홍차, 우유, 꿀',
 '음료, 과일, 얼음, 시럽',
 '음료, 레몬, 설탕, 탄산수',
 '음료, 녹차, 꿀, 레몬',
 '제과제빵, 밀가루, 버터, 설탕',
 '제과제빵, 계란, 밀가루, 우유',
 '제과제빵, 초콜릿, 생크림, 딸기',
 '제과제빵, 바나나, 계란, 설탕',
 '제과제빵, 아몬드, 설탕, 버터',
 '죽수프, 쌀, 닭고기, 참기름',
 '죽수프, 감자, 버터, 우유',
 '죽수프, 호박, 생크림, 소금',
 '죽수프, 해물, 파, 마늘',
 '죽수프, 버섯, 소고기, 우유',
 '축산물, 소고기, 양파, 마늘',
 '축산물, 돼지고기, 된장, 고추장',
 '축산물, 양고기, 허브, 올리브유',
 '축산물, 닭고기, 간장, 마늘',
 '축산물, 베이컨, 계란, 감자'
]


In [ ]:
import json

data_to_save = []

for qu in query_li:
    category = qu.split(', ')[0].strip()

    most_similar = find_similar(qu)
    # print(most_similar)

    response = gen_response(qu, most_similar, category)
    response = response.content
    print(response)

    response = response.split('A:')[-1].strip()
    matches = re.findall(r'[가-힣a-zA-Z0-9]+:\s*\d+%', response)
    A = ', '.join(matches)

    missed_q = []
    for q in qu.split(', ')[1:]:
        if q.strip() not in A:
            missed_q.append(q)

    if len(missed_q) != 0:
        response = gen_response_v2(A, missed_q)
        response = response.content

    print(response)

    data_to_save.append({
        "query": qu,
        "most_similar": most_similar,
        "response": response,
    })

with open('result_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_to_save, json_file, ensure_ascii=False, indent=4)

print("JSON 파일 저장이 완료되었습니다.")

# 평균 생성 시간 - 12초

/tmp/ipykernel_3229947/1013944166.py:20: DtypeWarning: Columns (20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(metadata_file)
Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been use

A: 김치: 20%, 돼지고기: 30%, 쌀: 25%, 마늘쫑: 10%, 절임무: 5%, 조미김: 5%, 지리멸치: 5%
A: 김치: 18%, 돼지고기: 27%, 마라: 5%, 쌀: 23%, 마늘쫑: 9%, 절임무: 4%, 조미김: 4%, 지리멸치: 4%.


/tmp/ipykernel_3229947/1013944166.py:20: DtypeWarning: Columns (20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(metadata_file)
Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been use

A: 1. 밥류의 요리 만들기:
    간장: 20%, 달걀: 5%, 대파: 3%, 올레오레진파프리카: 10%, 흑후추분말: 2%, 정제소금: 2%, 쌀: 40%, 대두단백: 5%, 백설탕: 5%, 볶음참깨: 2%, 숯불양념소스: 3%, 조각가슴살: 10%, 양배추: 5%, 양파: 5%, 옥수수전분: 2%, 냉동마늘: 2%, 당근: 5%, 로스티드치킨엑기스: 2%, 대파: 2%, 참기름: 3%, 매콤직화구이맛소스: 2%, 5리보뉴클레오티드나트륨: 1%, 대두유: 2%, 찹쌀: 10%, 양조간장: 2%

    이 배합비는 모든 재료를 혼합하여 맛있는 밥류 요리를 만드는 데 사용할 수 있습니다. 맛을 조절하고 필요에 따라 추가 재료로 맞춤화하세요.
1. 밥류의 요리 만들기:
    간장: 20%, 달걀: 5%, 대파: 3%, 올레오레진파프리카: 10%, 흑후추분말: 2%, 정제소금: 2%, 쌀: 40%, 대두단백: 5%, 백설탕: 5%, 볶음참깨: 2%, 숯불양념소스: 3%, 조각가슴살: 10%, 양배추: 5%, 양파: 5%, 옥수수전분: 2%, 냉동마늘: 2%, 당근: 5%, 로스티드치킨엑기스: 2%, 대파: 2%, 참기름: 3%, 매콤직화구이맛소스: 2%, 5리보뉴클레오티드나트륨: 1%, 대두유: 2%, 찹쌀: 10%, 양조간장: 2%

    이 배합비는 모든 재료를 혼합하여 맛있는 밥류 요리를 만드는 데 사용할 수 있습니다. 맛을 조절하고 필요에 따라 추가 재료로 맞춤화하세요.


/tmp/ipykernel_3229947/1013944166.py:20: DtypeWarning: Columns (20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(metadata_file)
Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been use

...

A: 양고기: 50%, 프리믹스-SP: 20%, 카라멜당시럽: 10%, 빵가루: 10%, 올리브유: 5%, 허브: 5%
양고기: 50%, 프리믹스-SP: 20%, 카라멜당시럽: 10%, 빵가루: 10%, 올리브유: 5%, 허브: 5%


/tmp/ipykernel_3229947/1013944166.py:20: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(metadata_file)
Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling pa

A: 닭고기: 50%, 간장: 10%, 마늘: 5%, 콩식용유: 10%, 닭가슴살: 10%, 폴리인산나트륨: 5%, 시즈닝dy: 5%, 정제소금: 5%, 난백분: 2%, 올리브유: 3%
닭고기: 50%, 간장: 10%, 마늘: 5%, 콩식용유: 10%, 닭가슴살: 10%, 폴리인산나트륨: 5%, 시즈닝dy: 5%, 정제소금: 5%, 난백분: 2%, 올리브유: 3%


/tmp/ipykernel_3229947/1013944166.py:20: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(metadata_file)
Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling pa

A: 베이컨: 25%, 계란: 10%, 감자: 30%, 부추: 5%, 양배추: 10%, 당근: 5%, 대파: 5%, 참기름: 5%
베이컨: 25%, 계란: 10%, 감자: 30%, 부추: 5%, 양배추: 10%, 당근: 5%, 대파: 5%, 참기름: 5%
JSON 파일 저장이 완료되었습니다.
